In [1]:
# Credits: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py


from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 10
epochs = 15

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Model 1: 3 conv layers

In [4]:
# https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

model1 = Sequential()

model1.add(Conv2D(32, kernel_size=(2, 2),
                 activation='relu',
                 input_shape=input_shape))
#model1.add(Conv2D(32, (2, 2), activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Conv2D(64, kernel_size=(2, 2),
                 activation='relu'))
#model1.add(Conv2D(64, (2, 2), activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Conv2D(128, kernel_size=(2, 2),
                 activation='relu'))
#model1.add(Conv2D(128, (2, 2), activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax'))

model1.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 27, 32)        160       
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 26, 26, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 26, 26, 64)        0    

In [5]:
model1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model1.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 727s 12ms/step - loss: 0.4172 - acc: 0.8958 - val_loss: 0.2513 - val_acc: 0.9566
Epoch 2/15
60000/60000 [==============================] - 725s 12ms/step - loss: 0.2180 - acc: 0.9469 - val_loss: 0.1665 - val_acc: 0.9619
Epoch 3/15
60000/60000 [==============================] - 726s 12ms/step - loss: 0.1772 - acc: 0.9586 - val_loss: 0.1513 - val_acc: 0.9787
Epoch 4/15
60000/60000 [==============================] - 731s 12ms/step - loss: 0.1464 - acc: 0.9660 - val_loss: 0.1041 - val_acc: 0.9822
Epoch 5/15
60000/60000 [==============================] - 728s 12ms/step - loss: 0.1259 - acc: 0.9698 - val_loss: 0.0906 - val_acc: 0.9768
Epoch 6/15
60000/60000 [==============================] - 735s 12ms/step - loss: 0.1171 - acc: 0.9716 - val_loss: 0.0744 - val_acc: 0.9826
Epoch 7/15
60000/6

# Model 2: 5 conv layers

In [4]:
# https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

model2 = Sequential()

model2.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#model2.add(Conv2D(8, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Dense(32, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model2.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu'))
#model2.add(Conv2D(16, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Dense(32, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))
#model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Dense(64, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model2.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu'))
#model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Dense(64, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model2.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu'))
#model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))

model2.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 8)         32        
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 8)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 26, 26, 32)        288       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        4624      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 16)        64 

In [5]:
model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 581s 10ms/step - loss: 0.2857 - acc: 0.9177 - val_loss: 0.0949 - val_acc: 0.9781
Epoch 2/15
60000/60000 [==============================] - 583s 10ms/step - loss: 0.1173 - acc: 0.9678 - val_loss: 0.0454 - val_acc: 0.9876
Epoch 3/15
60000/60000 [==============================] - 583s 10ms/step - loss: 0.0846 - acc: 0.9772 - val_loss: 0.0569 - val_acc: 0.9844
Epoch 4/15
60000/60000 [==============================] - 578s 10ms/step - loss: 0.0688 - acc: 0.9807 - val_loss: 0.0355 - val_acc: 0.9903
Epoch 5/15
60000/60000 [==============================] - 579s 10ms/step - loss: 0.0584 - acc: 0.9844 - val_loss: 0.0527 - val_acc: 0.9884
Epoch 6/15
60000/60000 [==============================] - 581s 10ms/step - loss: 0.0529 - acc: 0.9850 - val_loss: 0.0358 - val_acc: 0.9898
Epoch 7/15
60000/6

# Model 3: 7 conv layers

In [4]:
# https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

model3 = Sequential()

model3.add(Conv2D(8, kernel_size=(2, 2),
                 activation='relu',
                 input_shape=input_shape))
#model3.add(Conv2D(8, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(8, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(16, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(16, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(32, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(32, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(64, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(64, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(128, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(128, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(128, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(256, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(256, (5, 5), activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Dense(256, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))

model3.add(Conv2D(512, kernel_size=(5, 5),
                 activation='relu'))
#model3.add(Conv2D(512, (5, 5), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(BatchNormalization())
model3.add(Dropout(0.25))

model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))

model3.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 27, 8)         40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 8)         32        
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 27, 8)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 27, 27, 8)         72        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 16)        3216      
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 16)        64 

In [5]:
model3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model3.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 1716s 29ms/step - loss: 0.4542 - acc: 0.8610 - val_loss: 0.6839 - val_acc: 0.8987
Epoch 2/15
60000/60000 [==============================] - 1727s 29ms/step - loss: 0.1211 - acc: 0.9655 - val_loss: 0.1360 - val_acc: 0.9654
Epoch 3/15
60000/60000 [==============================] - 1726s 29ms/step - loss: 0.0884 - acc: 0.9739 - val_loss: 0.0833 - val_acc: 0.9793
Epoch 4/15
60000/60000 [==============================] - 1731s 29ms/step - loss: 0.0703 - acc: 0.9797 - val_loss: 0.0478 - val_acc: 0.9869
Epoch 5/15
60000/60000 [==============================] - 1699s 28ms/step - loss: 0.0599 - acc: 0.9826 - val_loss: 0.0609 - val_acc: 0.9846
Epoch 6/15
60000/60000 [==============================] - 1735s 29ms/step - loss: 0.0525 - acc: 0.9847 - val_loss: 0.0351 - val_acc: 0.9895
Epoch 7/15
6

# Conclusion

In [1]:
from prettytable import PrettyTable

In [2]:
x = PrettyTable()
x.field_names = ['Model', '# of Hidden Layers', 'Test Loss', 'Test Accuracy']
x.add_row(['Model 1', '3', '0.06815407012796959', '0.986'])
x.add_row(['Model 2', '5', '0.030686432317545177', '0.993'])
x.add_row(['Model 3', '7', '0.026872940180104297', '0.9927'])
print(x)

+---------+--------------------+----------------------+---------------+
|  Model  | # of Hidden Layers |      Test Loss       | Test Accuracy |
+---------+--------------------+----------------------+---------------+
| Model 1 |         3          | 0.06815407012796959  |     0.986     |
| Model 2 |         5          | 0.030686432317545177 |     0.993     |
| Model 3 |         7          | 0.026872940180104297 |     0.9927    |
+---------+--------------------+----------------------+---------------+


# Summary

There are 3 models with 3 different no. of convolution layers. 

1. In Model 1, No. of hidden layers are 3 with Test loss = 0.0681 and Test Accuracy = 98%
2. In Model 2, No. of hidden layers are 5 with Test loss = 0.0306 and Test Accuracy = 99.3%
3. In Model 3, No. of hidden layers are 7 with Test loss = 0.0268 and Test Accuracy = 99.27%

Model 3 has least loss and Model 2 has highest accuracy.